In [1]:
import numpy as np

# from keras.datasets import imdb
import tensorflow as tf
import tensorflow_datasets as tfds



In [2]:
import matplotlib.pyplot as plt


def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

In [3]:
dataset, info = tfds.load('imdb_reviews', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

train_dataset.element_spec

Dataset imdb_reviews downloaded and prepared to C:\Users\Drew\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [4]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [5]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [6]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

In [7]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i',
       'this', 'that', 'br', 'was', 'as', 'for', 'with', 'movie', 'but'],
      dtype='<U14')

In [44]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])
tf.keras.backend.clear_session()

In [45]:
print([layer.supports_masking for layer in model.layers])

[False, True, True, True, True]


In [46]:
sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
predictions = model.predict(np.array([sample_text]))
print(predictions[0])

[0.01530622]


In [47]:
padding = "the " * 2000
predictions = model.predict(np.array([sample_text, padding]))
print(predictions[0])

[0.01530622]


In [37]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [38]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)

Epoch 1/10
391/391 [==============================] - 559s 1s/step - loss: 0.6566 - accuracy: 0.5506 - val_loss: 0.5395 - val_accuracy: 0.7063
Epoch 2/10
391/391 [==============================] - 553s 1s/step - loss: 0.4115 - accuracy: 0.8077 - val_loss: 0.3789 - val_accuracy: 0.8245
Epoch 3/10
391/391 [==============================] - 502s 1s/step - loss: 0.3513 - accuracy: 0.8440 - val_loss: 0.3454 - val_accuracy: 0.8469
Epoch 4/10
391/391 [==============================] - 497s 1s/step - loss: 0.3302 - accuracy: 0.8576 - val_loss: 0.3346 - val_accuracy: 0.8552
Epoch 5/10
391/391 [==============================] - 483s 1s/step - loss: 0.3216 - accuracy: 0.8612 - val_loss: 0.3296 - val_accuracy: 0.8583
Epoch 6/10
391/391 [==============================] - 478s 1s/step - loss: 0.3179 - accuracy: 0.8644 - val_loss: 0.3311 - val_accuracy: 0.8620
Epoch 7/10
391/391 [==============================] - 475s 1s/step - loss: 0.3137 - accuracy: 0.8682 - val_loss: 0.3440 - val_accuracy: 0.8615

In [15]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

391/391 [==============================] - 88s 225ms/step - loss: 0.3153 - accuracy: 0.8579
Test Loss: 0.31534335017204285
Test Accuracy: 0.8578799962997437


In [42]:
sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
predictions = model.predict(np.array([sample_text]))
predictions

array([[0.4883921]], dtype=float32)

In [43]:
sample_text = ('i loved the movie')
redictions = model.predict(np.array([sample_text]))
predictions

array([[0.4883921]], dtype=float32)

In [20]:
model.save("C:\\Users\\Drew\\Documents\\app-school-450\\analysis\\saved_model_3.tf")

INFO:tensorflow:Assets written to: C:\Users\Drew\Documents\app-school-450\analysis\saved_model_3.tf\assets


INFO:tensorflow:Assets written to: C:\Users\Drew\Documents\app-school-450\analysis\saved_model_3.tf\assets
